In [1]:
import numpy as np
import pandas as pd
import random
import os
from datetime import datetime as dt

# Classifiers and regressors
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression

# Sklearn helper functions
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Data Loading
### Load the csv file from local

In [2]:
daily_data = pd.read_csv('data/van_weather_1990-01-01_2023-11-06.csv')
display(daily_data.head())
print(daily_data.columns)

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,1990-01-01,51.0,5.7095,2.0095,3.903249,1.759364,-2.227180,0.155060,0,0,1.600000,1.600000,0.00,7.0,18.250260,35.280000,222.90671,4.09,0.504341
1,1990-01-02,71.0,3.0595,0.0095,1.959500,-0.347375,-2.789511,-1.547468,0,0,1.000000,0.700000,0.21,5.0,12.261158,28.800000,171.74966,3.29,0.467342
2,1990-01-03,73.0,4.4095,1.9595,2.934500,1.055596,-1.572106,-0.585330,0,0,11.400002,10.600001,0.56,24.0,17.555307,34.560000,141.70381,1.89,0.201307
3,1990-01-04,61.0,7.3595,3.2095,4.917833,3.778481,0.573077,1.792292,0,0,12.599999,12.599999,0.00,14.0,18.806337,38.160000,162.86450,2.43,0.277709
4,1990-01-05,63.0,8.3595,3.6595,5.892833,4.432674,0.000019,2.062605,0,0,17.699999,17.699999,0.00,17.0,32.919827,64.439995,159.80486,0.64,0.168201


Index(['date', 'weather_code', 'temperature_2m_max', 'temperature_2m_min',
       'temperature_2m_mean', 'apparent_temperature_max',
       'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise',
       'sunset', 'precipitation_sum', 'rain_sum', 'snowfall_sum',
       'precipitation_hours', 'wind_speed_10m_max', 'wind_gusts_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration'],
      dtype='object')


# Data Transformation
## Transform the month data with cyclical encoding

In [3]:
# Using ordinal encoding  or leaving the month as a number may not be the best representation of month. 
# Since month is a cyclical data, we get inspiration from this work and use cyclical encoding: 
# https://www.kaggle.com/code/avanwyk/encoding-cyclical-features-for-deep-learning
# This preprocessing does not break golden rule because the max value of months is 12 as per common knowledge. 

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

daily_data['date'] = pd.to_datetime(daily_data['date'])
daily_data['month'] = daily_data['date'].dt.month
daily_data = encode(daily_data, 'month', 12)

## Select which columns to use and split the train and test

In [4]:
Usable_cols = ['temperature_2m_mean', 'wind_speed_10m_max', 
                'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration', 
               'precipitation_sum', 'month_sin', 'month_cos']
df = daily_data[Usable_cols].copy()
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=522)

X_train = train_df.drop(columns=['precipitation_sum'])
y_train = train_df['precipitation_sum']
X_test = test_df.drop(columns=['precipitation_sum'])
y_test = test_df['precipitation_sum']

X_train.head()

,temperature_2m_mean,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,month_sin,month_cos
4428,3.924084,7.594207,7.625911,10.56,1.169531,8.660254e-01,5.000000e-01
1182,6.765750,7.636753,144.196550,18.07,2.278618,1.000000e+00,6.123234e-17
12221,12.353417,21.398056,272.093350,26.07,3.796273,1.224647e-16,-1.000000e+00
4197,14.947000,8.311245,235.980600,31.24,4.915466,1.224647e-16,-1.000000e+00
7352,5.453250,12.758432,349.729030,11.33,1.357648,8.660254e-01,5.000000e-01


## Define the mean standard cross validate scores function

In [5]:
# Defining the function to calculate score for cross validation (credit to DSCI571)
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores.iloc[i], std_scores.iloc[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

# Define the models needed for regressor

In [6]:
# All the features are numerical features to be standardized. Here, we will use standard scalar.
preprocess = StandardScaler()

# We will evaluate a set of models for this project
regressor_models = {
    "Dummy Regressor": DummyRegressor(), 
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=522),
    "KNN Regressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "Ridge": Ridge(random_state=522)
}

cv_results = {}

for model_name, model in regressor_models.items():
    pipe = make_pipeline(preprocess, model)
    cv_results[model_name] = mean_std_cross_val_scores(pipe, X_train, y_train, return_train_score=True)
    print("Done with", model_name)

results_df = pd.DataFrame(cv_results).T
results_df

Done with Dummy Regressor
Done with Decision Tree Regressor
Done with KNN Regressor
Done with SVR
Done with Ridge


,fit_time,score_time,test_score,train_score
Dummy Regressor,0.001 (+/- 0.000),0.000 (+/- 0.000),-0.001 (+/- 0.002),0.000 (+/- 0.000)
Decision Tree Regressor,0.045 (+/- 0.001),0.001 (+/- 0.000),0.393 (+/- 0.034),1.000 (+/- 0.000)
KNN Regressor,0.004 (+/- 0.000),0.011 (+/- 0.000),0.596 (+/- 0.029),0.729 (+/- 0.004)
SVR,1.028 (+/- 0.034),0.388 (+/- 0.004),0.606 (+/- 0.025),0.609 (+/- 0.006)
Ridge,0.002 (+/- 0.000),0.001 (+/- 0.000),0.488 (+/- 0.018),0.488 (+/- 0.004)


Based on the performance above, SVR and KNN seems to outperfom than other models. Hence we decide to do the hyperparameter optimization on both models.

# KNN Regressor Hyperparameter Optimization

In [7]:
param_grid = {
    'kneighborsregressor__n_neighbors': [5, 10, 50, 100],
    'kneighborsregressor__weights': ['uniform', 'distance'],
    'kneighborsregressor__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'kneighborsregressor__leaf_size': [10, 30, 50, 100, 250, 500],
    'kneighborsregressor__p': [1, 2]
}

knn_pipe = make_pipeline(preprocess, KNeighborsRegressor())

knn_random_search = RandomizedSearchCV(knn_pipe, param_distributions=param_grid, n_iter=50, n_jobs=-1, return_train_score=True)
knn_random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kneighborsregressor',
                                              KNeighborsRegressor())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'kneighborsregressor__algorithm': ['auto',
                                                                           'ball_tree',
                                                                           'kd_tree',
                                                                           'brute'],
                                        'kneighborsregressor__leaf_size': [10,
                                                                           30,
                                                                           50,
                                                                           100,
                                                                           250,
                                                                           500],
                                        'kneighborsregressor__n_neighbors': [5,
                                                                             10,
                                                                             50,
                                                                             100],
                                        'kneighborsregressor__p': [1, 2],
                                        'kneighborsregressor__weights': ['uniform',
                                                                         'distance']},
                   return_train_score=True)

In [8]:
pd.DataFrame(knn_random_search.cv_results_)[
    [
        "mean_test_score",
        "param_kneighborsregressor__n_neighbors", 
        "param_kneighborsregressor__weights",
        "param_kneighborsregressor__algorithm", 
        "param_kneighborsregressor__leaf_size",
        "param_kneighborsregressor__p",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

rank_test_score,1,1,3,3,3,3,7,7,7,7,...,41,41,43,43,43,43,43,48,49,49
mean_test_score,0.637929,0.637929,0.63273,0.63273,0.63273,0.63273,0.630815,0.630815,0.630815,0.630815,...,0.596638,0.596638,0.595806,0.595806,0.595806,0.595806,0.595806,0.589018,0.576529,0.576529
param_kneighborsregressor__n_neighbors,10,10,10,10,10,10,50,50,50,50,...,50,50,5,5,5,5,5,100,100,100
param_kneighborsregressor__weights,distance,distance,uniform,uniform,uniform,uniform,distance,distance,distance,distance,...,uniform,uniform,uniform,uniform,uniform,uniform,uniform,distance,uniform,uniform
param_kneighborsregressor__algorithm,kd_tree,auto,auto,auto,auto,auto,kd_tree,auto,auto,brute,...,ball_tree,kd_tree,auto,auto,brute,kd_tree,kd_tree,kd_tree,auto,kd_tree
param_kneighborsregressor__leaf_size,30,50,30,100,250,10,500,250,50,10,...,30,50,30,250,30,30,50,30,250,10
param_kneighborsregressor__p,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
mean_fit_time,0.007119,0.006376,0.005869,0.008279,0.003252,0.004374,0.007578,0.002897,0.0039,0.003616,...,0.006,0.00868,0.004046,0.00455,0.004216,0.0105,0.004772,0.004597,0.006055,0.009117


In [9]:
print('KNN Best Parameters:')
print(knn_random_search.best_params_)
print()
print('KNN Best Score:')
print(knn_random_search.best_score_)

KNN Best Parameters:
{'kneighborsregressor__weights': 'distance', 'kneighborsregressor__p': 1, 'kneighborsregressor__n_neighbors': 10, 'kneighborsregressor__leaf_size': 50, 'kneighborsregressor__algorithm': 'auto'}

KNN Best Score:
0.6379292857402181


# SVR Hyperparameter Optimization

In [10]:
param_grid  = {
    'svr__C': np.logspace(-4, 3, 8),
    'svr__epsilon': np.logspace(-4, -1, 4),
    'svr__gamma': np.logspace(-3, 2, 6)
}

svr_pipe = make_pipeline(preprocess, SVR())

svr_random_search = RandomizedSearchCV(svr_pipe, param_distributions=param_grid, n_iter=50, n_jobs=-1, return_train_score=True)
svr_random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('svr', SVR())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'svr__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'svr__epsilon': array([0.0001, 0.001 , 0.01  , 0.1   ]),
                                        'svr__gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
                   return_train_score=True)

In [11]:
pd.DataFrame(svr_random_search.cv_results_)[
    [
        "mean_test_score",
        "param_svr__C", 
        "param_svr__epsilon", 
        "param_svr__gamma",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

rank_test_score,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
mean_test_score,0.689687,0.675383,0.645259,0.633757,0.613437,0.600894,0.599618,0.569462,0.567262,0.567225,...,-0.276627,-0.277257,-0.280391,-0.280428,-0.280499,-0.280877,-0.280892,-0.2809,-0.280924,-0.280935
param_svr__C,1000.0,100.0,10.0,1000.0,100.0,1.0,1.0,10.0,1000.0,1000.0,...,0.001,0.0001,0.0001,0.1,0.001,0.0001,0.0001,0.0001,0.001,0.01
param_svr__epsilon,0.0001,0.001,0.001,0.0001,0.01,0.1,0.001,0.0001,0.001,0.0001,...,0.001,0.01,0.1,0.0001,0.001,0.001,0.1,0.0001,0.001,0.001
param_svr__gamma,0.1,0.1,0.1,0.01,0.01,0.1,0.1,0.01,0.001,0.001,...,0.001,0.01,0.001,100.0,10.0,10.0,100.0,100.0,100.0,100.0
mean_fit_time,108.627108,10.339996,2.561906,7.601315,2.264166,1.312138,1.671424,1.705065,2.533919,2.525509,...,1.779079,1.670592,1.667284,2.064319,1.573578,1.561053,1.830051,1.876758,2.012012,2.035525


In [12]:
print('SVR Best Parameters:')
print(svr_random_search.best_params_)
print()
print('SVR Best Score:')
print(svr_random_search.best_score_)

SVR Best Parameters:
{'svr__gamma': 0.1, 'svr__epsilon': 0.0001, 'svr__C': 1000.0}

SVR Best Score:
0.6896866131811811


# Evaluate the best model

In [13]:
best_pipe = make_pipeline(preprocess, SVR(gamma=0.1, epsilon=0.01, C=100))
best_pipe.fit(X_train, y_train)
best_pipe.score(X_test, y_test)

0.6861948291515325